<a href="https://colab.research.google.com/github/cibergus/ML-open/blob/main/Exercise_2_MNIST_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms

torch.manual_seed(42);

In [2]:
# Design NN model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.features = nn.Sequential(
                            nn.Conv2d(1, 32, 3, 1),
                            nn.ReLU(),
                            nn.Conv2d(32, 64, 3, 1),
                            nn.ReLU(),
                            nn.MaxPool2d(2),
                            nn.Dropout(0.25),
                            nn.Flatten()
                        )
        self.classifier = nn.Sequential(
                            nn.Linear(9216, 128),
                            nn.ReLU(),
                            nn.Dropout(0.5),
                            nn.Linear(128, 10),
                            nn.LogSoftmax(1)
                        )
    
    def forward(self, x):
        x = self.features(x)
        output =  self.classifier(x)
        return output

In [3]:
# Create NN
model=Net()

In [4]:
# Load and Pre-process Data
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
trainset = datasets.MNIST('../data', train=True, download=True,
                    transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, 
                                           batch_size=64, 
                                           shuffle=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [5]:
# Function to test model
def test_model(model):
    # Load Test Data
    testset = datasets.MNIST('../data', train=False,
                        transform=transform)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=1000)

    model.eval()
    device = next(model.parameters()).device

    # Test model
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    # Print Accuracy
    print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:
# Training (3+5)
# 3 
device = 'cuda' if torch.cuda.is_available() else 'cpu'

loss_fn = nn.NLLLoss()
optimizer = optim.Adadelta(model.parameters(), lr=1.0)
model.train()
model=model.to(device)

for epoch in range(3):
    for batch_idx, (batch_data, batch_target) in enumerate(train_loader):
        # 5
        batch_data, batch_target = batch_data.to(device), batch_target.to(device)
        optimizer.zero_grad()
        batch_output = model(batch_data)
        loss = loss_fn(batch_output, batch_target)
        loss.backward()
        optimizer.step()
        # print results
        if batch_idx % 50 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch+1, batch_idx * len(batch_data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    test_model(model)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.302757
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.658285
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.337461
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.276276
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.291020
Train Epoch: 1 [16000/60000 (27%)]	Loss: 0.132165
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.068098
Train Epoch: 1 [22400/60000 (37%)]	Loss: 0.078086
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.142507
Train Epoch: 1 [28800/60000 (48%)]	Loss: 0.072709
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.135567
Train Epoch: 1 [35200/60000 (59%)]	Loss: 0.245092
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.050612
Train Epoch: 1 [41600/60000 (69%)]	Loss: 0.108775
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.059516
Train Epoch: 1 [48000/60000 (80%)]	Loss: 0.054293
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.103302
Train Epoch: 1 [54400/60000 (91%)]	Loss: 0.093048
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.118444

Test set: Accuracy: 9852/10000 (99%)

Train Epoch: 2 [0/60

In [7]:
# Save Model
torch.save(model.state_dict(), "mnist_cnn.pt")

### This notebook is a modified version of the PyTorch MNIST Example [here](https://github.com/pytorch/examples/tree/main/mnist).

Things to Try:
- Run the example on Github in a local or server development environment with an IDE like VSCode, PyCharm. Notice how the authors use argparse to pass in configuration parameters.
- Experiment with different NN model designs, loss functions, and optimizers.
- View the data for examples that fail. Is anything wrong with the data, or are they just difficult examples?
